In [1]:
from dataccess.nbfunctions import *
from dataccess import utils, query, mec, datashow
from dataccess.summarymetrics import plt, histogram, scatter
import config

from scipy.stats import pearsonr

%matplotlib inline

In [2]:
rm -r cache/xrd*

In [3]:
%pdb

Automatic pdb calling has been turned ON


In [4]:
MgO_ref_short = [get_query_dataset('material MgO runs 870 961 transmission 1. focal_size %d' % spotsize,
                                'MgO.ref.1C.short2.%sum' % spotsize, print_enable = False)
                                for spotsize in [4, 8, 18, 28, 38, 48, 58]]

In [5]:
# Exclude run 923 from the 58 um dataset due to its high number of events with zero signal
#MgO_ref_short_clean = [get_query_dataset('material MgO runs 870 961 transmission 1. focal_size %d' % spotsize,
#                                'MgO.ref.1C.short2.%sum' % spotsize, print_enable = False)
#                                for spotsize in [4, 8, 18, 38, 48]] + [query.DataSet([922], label = 'r922')]
#MgO_ref_short_clean.insert(0, get_query_dataset('runs 906 906', 'MgO.ref.1C.short2um', print_enable = False)) 

MgO_ref_short_clean = [get_query_dataset('material MgO runs 870 961 transmission 1. focal_size %d' % spotsize,
                                'MgO.ref.1C.short2.%sum' % spotsize, print_enable = False)
                                for spotsize in [4, 8, 18, 38, 48]] + [query.DataSet([922], label = 'r922')]

In [6]:
from dataccess import peakfinder


def peakfilter_frame(arr, detid = None, window_min = 0, radius = 4, thr_low = 20, thr_high = 50):
    """
    Mutates arr.
    """
    def filtfunc2(arr):
        base = 0#np.median(arr)
        return peakfinder.consolidate_peaks(arr, thr_low = thr_low, thr_high = thr_high, radius = radius)
    def process_island2(subarr):
        subarr[:] -= np.percentile(subarr, 20)
        subarr[:] = filtfunc2(subarr)
        return subarr
    
    if detid == 'quad2':
        clusters = clusters2
    elif detid == 'quad1':
        clusters = clusters1
    elif detid == 'si':
        return arr
    else:
        raise ValueError
    subregions = clustermap(arr, clusters, process_island2)
    for cluster, subarr in zip(clusters, subregions):
        bounding_view(arr, cluster)[:] = subarr
    arr[arr < window_min] = 0
    return arr

%run ../masks/masks.py
mask2 = np.load('../masks/quad2_mask_10.11.npy')
mask1 = np.load('../masks/quad1_mask_10.11.npy')
arr1, clusters1 = find_islands(mask1, 0, 1000)
arr2, clusters2 = find_islands(mask2, 0, 1000)

In [10]:
plot_xrd(MgO_ref_short[:2], compound_list = ['MgO'], normalization = None, bgsub = False, plot_patterns = True)

In [9]:
plot_xrd(MgO_ref_short[:2], compound_list = ['MgO'], normalization = 'peak',
         bgsub = True, plot_patterns = True,  detectors = ['quad2'])

In [7]:
progression = plot_xrd(MgO_ref_short_clean, compound_list = ['MgO'], normalization = 'peak',
        bgsub = False, plot_progression = True,  detectors = ['quad2', 'quad1'], 
        show_image = False, frame_processor = peakfilter_frame)

In [8]:
def set_param(progression, param, value, peak_index):
    for p in progression.patterns:
        peaks = p.peaks.peaks
        peaks[peak_index].param_values[param] = value
        peaks[peak_index].fixed_params[param] = True

def xrd_param(xrdset):
    for p in xrdset.patterns:
        peaks = p.peaks.peaks
        yield [pk.param_values for pk in peaks]
        
def mean_params(xrdset):
    params = list(xrd_param(xrdset))
    keys = params[0][0].keys()
    result = {}
    def get_mean(key):
        return np.mean([[p[key] for p in pattern] for pattern in params], axis = 0)
    return {k: get_mean(k) for k in keys} 

def print_param(xrdset, param):
    for p in xrdset.patterns:
        peaks = p.peaks.peaks
        print [pk.param_values[param] for pk in peaks]
        
def expand_mean_params(mean_dict, param, peakindex, numpeaks):
    value = lambda i: mean_dict[param][i]
    return [{param: value(i)} for i in range(numpeaks)]

In [9]:
#[[pk['amp'] for pk in row] for row in list(print_xrd_param(progression))]

In [10]:
mean_params(progression)

{'amp': array([  4.02752416,  41.65622548,  12.30466164,   8.35720697]),
 'cen': array([  33.47836818,   38.88925484,   55.97570466,  129.73735063]),
 'wid': array([ 0.08740687,  0.07775628,  0.11636086,  0.45057465])}

In [11]:
set_param(progression, 'wid',  0.45057465, 3)

In [12]:
mean_params(progression)

{'amp': array([  4.02752416,  41.65622548,  12.30466164,   8.35720697]),
 'cen': array([  33.47836818,   38.88925484,   55.97570466,  129.73735063]),
 'wid': array([ 0.08740687,  0.07775628,  0.11636086,  0.45057465])}

In [13]:
print_param(progression, 'wid')

[0.080558477351763261, 0.083463257279224531, 0.095592969068157493, 0.45057465]
[0.083463655048030397, 0.084265358548680475, 0.1202486020056861, 0.45057465]
[0.082365135547613266, 0.077215037477992285, 0.10659309248264977, 0.45057465]
[0.084286703705762619, 0.069567473487707393, 0.11155034295847482, 0.45057465]
[0.097289936120168535, 0.082956809591578182, 0.14347272877284725, 0.45057465]
[0.096477323856886149, 0.069069764202416686, 0.12070740658431932, 0.45057465]


In [36]:
start_params = expand_mean_params(mean_params(progression), 'wid', -1, 4)
start_params

[{'wid': 0.087406871938370698},
 {'wid': 0.077756283431266601},
 {'wid': 0.11636085697868913},
 {'wid': 0.45057465270181857}]

In [37]:
fixed = [['wid'] for _ in range(len(progression.patterns))]

In [14]:
progression = plot_xrd(MgO_ref_short_clean, x = progression, compound_list = ['MgO'], normalization = 'peak',
        bgsub = False, plot_progression = True,  detectors = ['quad2', 'quad1'], 
        show_image = False, frame_processor = peakfilter_frame)

In [43]:
mean_params(progression)

{'amp': array([  4.06479575,  41.73453223,  12.46360435,   0.83303204]),
 'cen': array([  33.47863548,   38.88918515,   55.9755274 ,  129.85774284]),
 'wid': array([ 0.08740687,  0.07775628,  0.11636086,  0.08740687])}

In [15]:
from dataccess import geometry

In [7]:
def test_add_patterns():
    d1, d2 = map(lambda d: xrd.Pattern(xrd.XRDset(d, 'quad2', ['MgO'])), MgO_ref_short_clean[:2])
    d3 = d1 + d2
    assert d3 is not None
    assert np.shape(d3.angles) == np.shape(d3.intensities)

In [9]:
def my_200_array_new(arr, bgsub = False, **kwargs):
    dss = xrd.Pattern.from_dataset(arr, 'quad2', ['MgO'], label  = 'test')
    return dss.peak_sizes()
    #angles, intensities, _ = xrd.process_dataset(dss, bgsub = bgsub)
    #return xrd.peak_sizes(angles, intensities, dss.compound_list[0])[1]

In [9]:
def my_200_array(arr, bgsub = False, **kwargs):
    dss = xrd.Pattern(arr, 'quad2', ['MgO'])
    return dss.peak_sizes()
    #angles, intensities, _ = xrd.process_dataset(dss, bgsub = bgsub)
    #return xrd.peak_sizes(angles, intensities, dss.compound_list[0])[1]

In [23]:
d1.get_attribute('focal_size')

2.0

In [22]:
d1 = get_query_dataset('material MgO runs 870 961 transmission 1. focal_size %d' % 2,
                                'MgO.ref.1C.short2.%sum' % 2, print_enable = False)

In [29]:
d1, d2 = MgO_ref_short_clean[:2]

In [37]:
np.mean(d1.evaluate('quad2').mean)

-68.242721825892659

In [11]:
arr = d1.evaluate('quad2').mean

In [16]:
peakarr = my_200_array_new(arr)

In [18]:
np.isclose(peakarr, np.array([  1.40817828,  28.70672704, -59.59338818])).all()

True

In [6]:
def get_query_dataset(querystring, alias = None, print_enable = True, **kwargs):
    dataset = query.main(querystring.split(), label = alias, **kwargs)
    if utils.isroot() and print_enable:
        print "Runs: ", dataset.runs
    return dataset

In [25]:
def test_peak_sizes():
    d1 = get_query_dataset('runs 906 906')
    arr = d1.evaluate('quad2').mean
    assert np.isclose(np.mean(arr), -68.242721825892659)
    peakarr = my_200_array_new(arr)
    assert np.isclose(peakarr, np.array([  1.41028554,  28.32860824, -69.18903552])).all()

In [28]:
test_peak_sizes()

Runs:  [906]


AssertionError: 

> <ipython-input-25-66c325c5b0fc>(4)test_peak_sizes()
      2     d1 = get_query_dataset('runs 906 906')
      3     arr = d1.evaluate('quad2').mean
----> 4     assert np.isclose(np.mean(arr), -68.242721825892659)
      5     peakarr = my_200_array_new(arr)
      6     assert np.isclose(peakarr, np.array([  1.41028554,  28.32860824, -69.18903552])).all()

ipdb> np.mean(arr)
-45.090074155928058
ipdb> c


In [ ]:
plot_xrd(MgO_ref_short_clean[:2], compound_list = ['MgO'], detectors = ['quad2'], 
         normalization = 'peak', bgsub = False, plot_patterns = True, 
         frame_processor = peakfilter_frame, show_image = False)

IndexError: index 0 is out of bounds for axis 0 with size 0

> /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dataccess-1.0-py2.7.egg/dataccess/xrd.py(552)get_normalization()
    550         raise NotImplementedError
    551     elif type == 'peak': # Normalize by size of first peak
--> 552         return np.array([pat.peak_sizes(ax = ax, fixed_params = fixed_params)[0] for pat in patterns])
    553     else: # Interpret type as the name of a function in config.py
    554         try:

ipdb> peaks
*** NameError: name 'peaks' is not defined
ipdb> up
> /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dataccess-1.0-py2.7.egg/dataccess/xrd.py(430)plot()
    428         if normalization:
    429             scale = get_normalization([ self ], type = normalization, ax = ax,
--> 430                     fixed_params = fixed_params)[0]
    431         else:
    432             scale = 1.

ipdb> peaks
*** NameError: name 'peaks' is not defined
ipdb> down
> /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-pack

In [5]:
plot_xrd([MgO_ref_short_clean[0]], compound_list = ['MgO'], detectors = ['quad2', 'quad1'], 
         normalization = 'peak', bgsub = False, plot_patterns = True, 
         frame_processor = peakfilter_frame, show_image = False, fixed_params = ['wid'])

[29] > /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dataccess-1.0-py2.7.egg/dataccess/xrd.py(359)_peak_size_gaussian_fit()
-> for name, p in pars.iteritems():
(Pdb++) p name
*** SyntaxError: invalid syntax (<stdin>, line 1)
(Pdb++) p
<Parameter 'wid', value=0.2 (fixed), bounds=[None:None]>
(Pdb++) name
'wid'
(Pdb++) fixed_params
['wid']
(Pdb++) c
[29] > /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dataccess-1.0-py2.7.egg/dataccess/xrd.py(359)_peak_size_gaussian_fit()
-> for name, p in pars.iteritems():
(Pdb++) c
[29] > /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dataccess-1.0-py2.7.egg/dataccess/xrd.py(359)_peak_size_gaussian_fit()
-> for name, p in pars.iteritems():
(Pdb++) c
[29] > /reg/neh/home/ohoidn/venv_test/mecana2/lib/python2.7/site-packages/dataccess-1.0-py2.7.egg/dataccess/xrd.py(359)_peak_size_gaussian_fit()
-> for name, p in pars.iteritems():
(Pdb++) c


In [7]:
%pdb

Automatic pdb calling has been turned ON
